## 1. Preprocressing

In [2]:
import numpy as np
from datascience import *

In [3]:
daily_casualties_gaza = Table.read_table('casualties_daily_gaza_2025-04-12.csv')
daily_casualties_gaza.show(10)

report_date,report_source,report_period,ext_massacres_cum,killed,killed_cum,ext_killed,ext_killed_cum,ext_killed_children_cum,ext_killed_women_cum,injured_cum,ext_injured,ext_injured_cum,ext_civdef_killed_cum,med_killed_cum,ext_med_killed_cum,press_killed_cum,ext_press_killed_cum
2023-10-07,mohtel,24,0,232,232,232,232,0,0,1610,1610,1610,0,6,6,1,1
2023-10-08,mohtel,24,0,138,370,138,370,78,41,1788,178,1788,0,nan,6,1,1
2023-10-09,mohtel,24,8,190,560,190,560,91,61,2271,483,2271,0,6,6,3,3
2023-10-10,mohtel,24,8,340,900,340,900,260,230,4000,1729,4000,0,nan,6,7,7
2023-10-11,gmotel,24,23,200,1100,200,1100,398,230,5184,1184,5184,0,10,10,nan,7
2023-10-12,mohtel,24,23,317,1417,317,1417,500,276,5763,579,5763,0,11,11,nan,7
2023-10-13,missing,24,23,483,1900,483,1900,500,392,nan,1475,7238,0,11,11,nan,7
2023-10-14,gmotel,24,23,328,2228,328,2228,600,600,8714,1476,8714,0,11,11,nan,7
2023-10-15,gmotel,24,23,442,2670,442,2670,700,735,9200,486,9200,0,nan,11,nan,7
2023-10-16,gmotel,24,371,138,2808,138,2808,853,936,10850,1650,10850,7,37,37,nan,7


In [6]:
daily_casualties_westbank = Table.read_table('casualties_daily_west_bank_2025-04-12.csv')
daily_casualties_westbank.show(5)

report_date,verified.killed,verified.killed_cum,verified.injured,verified.injured_cum,verified.killed_children,verified.killed_children_cum,verified.injured_children,verified.injured_children_cum,killed_cum,killed_children_cum,injured_cum,injured_children_cum,settler_attacks_cum,flash_source
2023-10-07,6,6,197,197,2,2,31,31,3,0,23,1,3,un
2023-10-08,8,14,127,324,3,5,7,38,13,1,194,1,9,un
2023-10-09,4,18,88,412,0,5,19,57,15,3,295,34,19,un
2023-10-10,5,23,72,484,0,5,7,64,19,3,332,39,29,un
2023-10-11,6,29,85,569,2,7,17,81,26,3,427,62,39,un


In [ ]:
pip install pdfplumber

In [1]:
import pdfplumber
import warnings
import logging

# To ignore the warnings
warnings.filterwarnings("ignore")
logging.getLogger("pdfminer").setLevel(logging.ERROR)

with pdfplumber.open('killed-in-gaza_moh_2025-03-23.pdf') as pdf:
    total_pages = len(pdf.pages)
    batch_size = 50  # process 50 pages at a time
    batch_number = 1
    
    for start_page in range(0, total_pages, batch_size):
        end_page = min(start_page + batch_size, total_pages)
        print(f"Processing pages {start_page+1} to {end_page}...")

        batch_rows = []
        columns = None

        for page_num in range(start_page, end_page):
            page = pdf.pages[page_num]
            table = page.extract_table()
            if table:
                if columns is None:
                    columns = table[0]  # first page -> header
                    batch_rows.extend(table[1:])  # first page -> skip header
                else:
                    batch_rows.extend(table)  # other pages -> take all rows

        if batch_rows:
            # Building a small Table for this batch
            batch_table = Table().with_columns(*[(col, []) for col in columns])

            for row in batch_rows:
                batch_table = batch_table.with_row(row)

            # Saving this batch separately
            filename = f'killed_in_gaza_batch_{batch_number}.csv'
            batch_table.to_csv(filename)

        batch_number += 1

Processing pages 1 to 50...
Processing pages 51 to 100...
Processing pages 101 to 150...
Processing pages 151 to 200...
Processing pages 201 to 250...
Processing pages 251 to 300...
Processing pages 301 to 350...
Processing pages 351 to 400...
Processing pages 401 to 450...
Processing pages 451 to 500...
Processing pages 501 to 550...
Processing pages 551 to 600...
Processing pages 601 to 650...
Processing pages 651 to 700...
Processing pages 701 to 750...
Processing pages 751 to 800...
Processing pages 801 to 850...
Processing pages 851 to 900...
Processing pages 901 to 950...
Processing pages 951 to 981...


In [44]:
import csv
import glob
import os

# Finding all CSV files
csv_files = sorted(glob.glob('killed_in_gaza_batch_*.csv'))

# Creating a file to store the combined output
with open('killed_in_gaza.csv', 'w', encoding='utf-8-sig', newline='') as output_file:
    writer = None

    for filename in csv_files:
        with open(filename, 'r', encoding='utf-8-sig') as f:
            reader = csv.reader(f)
            
            if writer is None:
                headers = next(reader)  # read header
                writer = csv.writer(output_file)
                writer.writerow(headers)

            for row in reader:
                writer.writerow(row)
        os.remove(filename)


In [8]:
killed_in_gaza = Table.read_table('killed_in_gaza.csv')
# killed_in_gaza.sort("Index", descending=True).show(5)
killed_in_gaza.show(5)

Index,Name,مس#ا,Born,Age,Sex,ID number
1,Mohammed Hani Mohammed Al-Zahhar,راهزلا دمحم يناه دمحم,2023-08-09,0,m,444196471
2,Nour Mohammed Aed Udwan,ناودع دئاع دمحم رون,2023-02-27,0,f,444484729
3,Khaled Mumin Amin Shabir,ريبش 6ما نمؤم دلاخ,2023-04-16,0,m,444799647
4,Yamen Ibrahim Farid Qaddoura,هرودق ديرف ميهاربا نماي,2023-06-10,0,m,445958408
5,Ahmed Rami Ahmed Abdel Qadir,رداقلا دبع دمحأ يمار دمحأ,2022-12-22,0,m,445808546


In [48]:
killed_in_gaza.num_rows

50020

In [66]:
import json

# Loading JSON
with open('infrastructure-damaged_2025-04-12.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Flattening records
flattened_data = []

for record in data:
    flat_record = {}
    for key, value in record.items():
        if isinstance(value, dict):
            for sub_key, sub_value in value.items():
                flat_record[f"{key}_{sub_key}"] = sub_value
        else:
            flat_record[key] = value
    flattened_data.append(flat_record)

# Finding all possible headers from all records
all_keys = set()
for row in flattened_data:
    all_keys.update(row.keys())

# headers = sorted(all_keys)

# Saving to CSV
with open('infrastructure_damaged.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.DictWriter(f, fieldnames=headers)
    writer.writeheader()
    
    for row in flattened_data:
        # filling missing keys with 0
        complete_row = {key: row.get(key, 0) for key in headers}
        writer.writerow(complete_row)


In [10]:
infrastructure_damaged = Table.read_table('infrastructure_damaged.csv')

# Making report_date the first column
labels = list(infrastructure_damaged.labels)
labels.remove('report_date')
new_order = ['report_date'] + labels  # Put it first

# Rearrange the Table
infrastructure_damaged = infrastructure_damaged.select(new_order)

infrastructure_damaged.show(5)

report_date,civic_buildings_destroyed,civic_buildings_ext_destroyed,educational_buildings_damaged,educational_buildings_destroyed,educational_buildings_ext_damaged,educational_buildings_ext_destroyed,places_of_worship_churches_destroyed,places_of_worship_ext_churches_destroyed,places_of_worship_ext_mosques_damaged,places_of_worship_ext_mosques_destroyed,places_of_worship_mosques_damaged,places_of_worship_mosques_destroyed,residential_destroyed,residential_ext_destroyed
2023-10-07,0,5,0,0,15,1,0,0,4,2,0,0,0,80
2023-10-08,0,11,0,0,30,1,0,0,8,4,0,0,159,159
2023-10-09,0,16,0,0,45,2,0,0,12,6,0,0,790,790
2023-10-10,0,22,0,0,60,2,0,0,17,8,0,0,1009,1009
2023-10-11,27,27,75,3,75,3,0,0,21,10,0,10,2835,2835


In [53]:
#function that check for any issue values in columns
def check_and_clean_column(table, column_name):
    print(f"checking column: {column_name}")
    
    #count nones
    none_count = table.where(column_name, are.equal_to(None)).num_rows
    print(f"  None values: {none_count}")

    #first check if column is numeric
    column_array = np.array(table.column(column_name))
    try:
        #check nans
        is_numeric = np.issubdtype(column_array.dtype, np.number)
        if is_numeric:
            nan_count = np.sum(np.isnan(column_array))
            valid_mask = ~np.isnan(column_array)
        else:
            # For non-numeric columns, just check for empty strings
            nan_count = np.sum(column_array == '')
            valid_mask = (column_array != '')
        
        print(f"  NaN or empty values: {nan_count}")
        
        #total bad values
        total_problematic = none_count + nan_count
        print(f"  Bad values: {total_problematic} out of {table.num_rows}")
        
        return {
            'column_name': column_name,
            'none_count': none_count,
            'nan_count': nan_count,
            'total_problematic': total_problematic,
            'valid_mask': valid_mask,
            'cleaned_array': column_array[valid_mask] if total_problematic > 0 else column_array
        }
    except Exception as e:
        print(f"  error in checking column: {e}")
        return {
            'column_name': column_name,
            'none_count': none_count,
            'nan_count': 0,
            'total_problematic': none_count,
            'valid_mask': np.ones(len(column_array), dtype=bool),
            'cleaned_array': column_array
        }


In [55]:
#check all datasets for issues
def check_all_datasets():
    print("----------- CHECKING GAZA CASUALTIES DATA -----------")
    gaza_columns_to_check = ['killed', 'killed_cum', 'injured_cum', 'ext_killed_children_cum', 'ext_killed_women_cum']
    gaza_column_info = {}
    
    for col in gaza_columns_to_check:
        if col in daily_casualties_gaza.labels:
            gaza_column_info[col] = check_and_clean_column(daily_casualties_gaza, col)
    
    print("\n----------- CHECKING WEST BANK CASUALTIES DATA -----------")
    wb_columns_to_check = ['verified.killed_cum', 'verified.injured_cum', 'verified.killed_children_cum']
    wb_column_info = {}
    
    for col in wb_columns_to_check:
        if col in daily_casualties_westbank.labels:
            wb_column_info[col] = check_and_clean_column(daily_casualties_westbank, col)
    
    print("\n----------- CHECKING INFRASTRUCTURE DATA -----------")
    infra_columns_to_check = ['residential_destroyed', 'educational_buildings_destroyed', 
                             'civic_buildings_destroyed', 'places_of_worship_mosques_destroyed']
    infra_column_info = {}
    
    for col in infra_columns_to_check:
        if col in infrastructure_damaged.labels:
            infra_column_info[col] = check_and_clean_column(infrastructure_damaged, col)
    
    print("\n----------- CHECKING KILLED IN GAZA DATA -----------")
    killed_columns_to_check = ['Age', 'Sex']
    killed_column_info = {}
    
    for col in killed_columns_to_check:
        if col in killed_in_gaza.labels:
            killed_column_info[col] = check_and_clean_column(killed_in_gaza, col)
    
    return gaza_column_info, wb_column_info, infra_column_info, killed_column_info

def check_column_types():
    print("\n----------- COLUMN DATA TYPES -----------")
    
    #gaza casualties ds
    print("gaza casualties key columns:")
    for col in ['report_date', 'killed_cum', 'ext_killed_children_cum', 'ext_killed_women_cum']:
        if col in daily_casualties_gaza.labels:
            try:
                value = daily_casualties_gaza.column(col).item(0)
                print(f"  {col}: {type(value)} (example: {value})")
            except:
                print(f"  {col}: Unable to determine type")
    
    #west bank casualties ds
    print("\nwest bank casualties key columns:")
    for col in ['report_date', 'verified.killed_cum', 'verified.injured_cum']:
        if col in daily_casualties_westbank.labels:
            try:
                value = daily_casualties_westbank.column(col).item(0)
                print(f"  {col}: {type(value)} (example: {value})")
            except:
                print(f"  {col}: Unable to determine type")
    
    #infrastructure ds
    print("\ninfrastructure key columns:")
    for col in ['report_date', 'residential_destroyed', 'educational_buildings_destroyed']:
        if col in infrastructure_damaged.labels:
            try:
                value = infrastructure_damaged.column(col).item(0)
                print(f"  {col}: {type(value)} (example: {value})")
            except:
                print(f"  {col}: Unable to determine type")
    
    #killed in gaza ds
    print("\killed in gaza key columns:")
    for col in ['Age', 'Sex']:
        if col in killed_in_gaza.labels:
            try:
                value = killed_in_gaza.column(col).item(0)
                print(f"  {col}: {type(value)} (example: {value})")
            except:
                print(f"  {col}: Unable to determine type")

# Run the data checking process
try:
    print("----------- STARTING DATA CHECKING -----------")
    
    #check for issues in all datasets
    gaza_column_info, wb_column_info, infra_column_info, killed_column_info = check_all_datasets()
    
    #check column data types
    check_column_types()
    
    print("\n----------- DATA CHECKING COMPLETE -----------")
    
except Exception as e:
    print(f"Error in data checking process: {e}")

----------- STARTING DATA CHECKING -----------
----------- CHECKING GAZA CASUALTIES DATA -----------
checking column: killed
  None values: 0
  NaN or empty values: 27
  Bad values: 27 out of 551
checking column: killed_cum
  None values: 0
  NaN or empty values: 7
  Bad values: 7 out of 551
checking column: injured_cum
  None values: 0
  NaN or empty values: 15
  Bad values: 15 out of 551
checking column: ext_killed_children_cum
  None values: 0
  NaN or empty values: 0
  Bad values: 0 out of 551
checking column: ext_killed_women_cum
  None values: 0
  NaN or empty values: 0
  Bad values: 0 out of 551

----------- CHECKING WEST BANK CASUALTIES DATA -----------
checking column: verified.killed_cum
  None values: 0
  NaN or empty values: 147
  Bad values: 147 out of 551
checking column: verified.injured_cum
  None values: 0
  NaN or empty values: 160
  Bad values: 160 out of 551
checking column: verified.killed_children_cum
  None values: 0
  NaN or empty values: 147
  Bad values: 147 o

<>:72: SyntaxWarning: invalid escape sequence '\k'
<>:72: SyntaxWarning: invalid escape sequence '\k'
/var/folders/6f/57h2c3g508nf1ps7ctmbbxtm0000gn/T/ipykernel_14024/3497494693.py:72: SyntaxWarning: invalid escape sequence '\k'
  print("\killed in gaza key columns:")


## 2. Exploratory Analysis & Visualization

In [ ]:
# casualities over time for gaza, westbank (overlaid line plot)

In [ ]:
# age groups (in ranges) histogram (killed in gaza)

In [ ]:
# join by report_date, number of building vs num killed in gaza (scatter)

In [ ]:
# group by gender (killed in gaza) bar chart

In [ ]:
# something for period of ceasefire vs normal casualities

In [ ]:
# select just medical killed column vs time (and press)

In [ ]:
# settler attacks vs casualities over time in westband

## 3. Hypothesis Testing

In [ ]:
# null: woman and men are being targetted equally
# alternative: woman are being targetted more/ children
# statistic: 

In [ ]:
# null: all ages are being targetted equally
# alternative: younger people (under a certain age)
# use td

In [ ]:
# 
# 
# 